Data Collocation

In [8]:
# taamolat data set
taamolat_directory  = 'Authors Dataset/taamolat/'
TAAMOLAT_NUMBER_OF_AUTHORS = 7
TAAMOLAT_NUMBER_OF_SAMPLES_PER_AUTHOR = 10

# alaraby data set
alaraby_directory = 'Authors Dataset/alaraby/'
ALARABY_NUMBER_OF_AUTHORS = 9
ALARABY_NUMBER_OF_SAMPLES_PER_AUTHOR = 10

Function for return the list of authors names for a specific dataset

In [9]:
def get_authors_names(directory_name, number_of_authors):
    import configparser

    # Create a ConfigParser object
    config = configparser.ConfigParser()

    # save the authors names in a list
    authors = []
    for i in range(number_of_authors):
        # Read the properties file with the correct encoding (e.g., 'utf-8')
        with open(f'{directory_name}a{i}/a{i}_0.properties', encoding='utf-8') as f:
            # Add a default section header to the file
            content = '[default]\n' + f.read()
            config.read_string(content)

        # Get the author name
        # Get the value of the 'author_name' key
        author_name = config.get('default', 'author_name')
        authors.append(author_name)

    return authors

In [10]:
taamolat_authors = get_authors_names(taamolat_directory, TAAMOLAT_NUMBER_OF_AUTHORS)
# manually add the arabic names of the authors of the taamolat dataset
taamolat_authors = ['محمد عواد', 'محمد زينوبة', 'يونس جعدي', 'نور الدين رافع', 'عبير قلوطي', 'معتز خطيب', 'إبراهيم نصر الله']
alaraby_authors = get_authors_names(alaraby_directory, ALARABY_NUMBER_OF_AUTHORS)

print(taamolat_authors[0])
print(alaraby_authors[0])


محمد عواد
عبد الله الشريف


Read the paragraphs of the authors 

In [11]:
# Get each paragraph with its author
def get_paragraphs_with_authors(directory_name, number_of_authors, number_of_samples_per_author, authors_names):
    paragraphs = []
    for i in range(number_of_authors):
        for j in range(number_of_samples_per_author):
            with open(f'{directory_name}a{i}/sample{j}.txt', encoding='utf-8') as file:
                current_paragraph = ''
    
                for line in file:
                    line = line.strip()
                
                    if line:  # If the line is not empty
                        current_paragraph += line + ' '
                    elif current_paragraph:  # If an empty line is encountered and there is a current paragraph
                        paragraphs.append((current_paragraph.strip(), authors_names[i]))
                        current_paragraph = ''

                if current_paragraph:  # Append the last paragraph if it exists after reaching the end of the file
                    paragraphs.append((current_paragraph.strip(), authors_names[i]))
    return paragraphs

In [23]:
taamolat_data = get_paragraphs_with_authors(taamolat_directory, TAAMOLAT_NUMBER_OF_AUTHORS, TAAMOLAT_NUMBER_OF_SAMPLES_PER_AUTHOR, taamolat_authors)
alaraby_data = get_paragraphs_with_authors(alaraby_directory, ALARABY_NUMBER_OF_AUTHORS, ALARABY_NUMBER_OF_SAMPLES_PER_AUTHOR, alaraby_authors)

# for paragraph in taamolat_data:
#     print(paragraph)
#     print()

# for paragraph in alaraby_data:
#     print(paragraph)
#     print()


# combine the two datasets
data_set = taamolat_data + alaraby_data
# make sure that the data is shuffled
import random
random.shuffle(data_set)

number_of_samples = len(data_set)


for paragraph in data_set:
    print(paragraph)
    print()

('غريب عالم الإبداع، بسيط في آن، ويدعو للدهشة. كيف يكتب واحد مثل نجيب محفوظ حوالي خمسين رواية، خمسين عالما، وهو لم ير من العالم بوضوح غير القاهرة؟ أين تختبئ تلك الشبكة العملاقة من الحيوات، وكيف تتفجر متألقة؟ وكيف ينشق هذا الهدوء، وهو هدوء ظاهري حتما، ويُعطي بامتياز كل هذا الألق الذي تخفيه الأعماق؟', 'إبراهيم نصر الله')

('وأشارت إلى أن نتنياهو سيهاجم الأحزاب اليمينية الصغيرة وسيتهمها بالتسبب في عدم تمكنه من تشكيل الحكومة لإقدامها على محاولة ابتزازه أثناء سعيه لبناء تحالفه بعد الانتخابات وطرح مطالب مبالغاً فيها، مشيرة إلى أن نتنياهو سيصف هذه الأحزاب بأنها تمثل خطراً على حكم اليمين.', 'صالح النعامي')

('في نهاية مضمار الحياة نتوج بالموت، كجائزة تعب، وذكرى مرور..', 'محمد زينوبة')

('نسوة المدينة قلنَ حاش لله ما علمنا عليه من سوء، لم يمكرن به، ولم يفترين ويكذبن. ذلك أن الأصل في قلوبهن الصدق والرأفة.', 'محمد زينوبة')

('وإذا كان الحديث السابق كله عن أنبياء، فقد تحدث القرآن أيضًا عن رغبات بعض الصحابة يوم أُحُد "حَتَّىٰ إِذَا فَشِلْتُمْ وَتَنَازَعْتُمْ فِي الْأَمْرِ وَعَصَيْتُم مِّن بَعْدِ مَ

Preprocessing The data sets

In [13]:

# Normalization: remove diacritics from the text & Punctuation mark removal & Numbers & Non-arabic & alef, teh 
def normalization(text):
    import re
    import tnkeeh
    from pyarabic.araby import strip_tashkeel, normalize_alef, normalize_teh
    text = re.sub(r'[^\u0621-\u064A\s]', '', text)
    text = strip_tashkeel(text)
    text = normalize_alef(text)
    text = normalize_teh(text)
    tnkeeh_obj = tnkeeh.Tnkeeh(
    remove_special_chars=True,
    remove_english=True, normalize=True, remove_diacritics=True,
    remove_tatweel=True, remove_html_elements=True,
    remove_links=True, remove_twitter_meta=True
    )
    text = tnkeeh_obj.clean_raw_text(text)[0]
    
    return text

text = "يْشَارْ إِلَasfgى ةهأغإلأألأَنْ اللُّغَ15+62ة الْعَ?!#@$#^رْبِيَّة"
text = normalization(text)
print(text)





يشار الا ههاغالاالان اللغه العربيه


In [14]:

# stemming
def stemming(word):
    from nltk.stem.isri import ISRIStemmer # special stemmer for arabic
    # Create an instance of the ISRI Arabic Stemmer

    stemmer = ISRIStemmer()
    return stemmer.stem(word)

text = 'المدرسة'
result = stemming(text)
print(result)



درس


In [15]:
# POS tagging
def pos_tagging(text):

    # from farasa.pos import FarasaPOSTagger
    # tagger = FarasaPOSTagger()
    # tags = tagger.tag_segments(text, combine_subtokens=True)
    # return tags

    # import json
    # import requests
    # api_key = "mgmfGsWLmIFJznbMaC"

    # url = 'https://farasa.qcri.org/webapi/pos/'
    # payload = {'text': text, 'api_key': api_key}
    # data = requests.post(url, data=payload)
    # result = json.loads(data.text)

    # result = result['text']

    # words_pos_tags = []
    # for obj in result:
    #     words_pos_tags.append((obj['surface'], obj['POS']))

    # # remove the first and last elements of the list
    # words_pos_tags = words_pos_tags[1:-1]

    # return words_pos_tags
    pass


text = 'يُشار إلى أن اللغة العربية'
result = pos_tagging(text)
print(result)






None


In [16]:
# tokenization
def tokenization(text):
    from pyarabic.araby import tokenize
    
    text = tokenize(text)
    return text

text = 'يُشار إلى أن اللغة العربية'

result = tokenization(text)
print(result)



['يُشار', 'إلى', 'أن', 'اللغة', 'العربية']


Arabic Stop Words

In [17]:
from nltk.corpus import stopwords

stop_words_file_name = 'stop_words.txt'
stop_words = []
with open(stop_words_file_name, encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        stop_words.append(line)

stop_words = list(set(stop_words))
stop_words += stopwords.words('arabic')

print(stop_words)

['', 'اللاتي', 'أهلا', 'بين', 'مع', 'وقد', 'شتانَ', 'وقالت', 'عندما', 'إليكَ', 'حجا', 'صراحة', 'ياء', 'أطعم', 'إليكنّ', 'سوى', 'اربعة', 'كأن', 'أولاء', 'وإن', 'يناير', 'وكل', 'سوف', 'جيم', 'إذاً', 'دواليك', 'صبرا', 'ظ', 'سنة', 'ست', 'عوض', 'غ', 'ح', 'ستمائة', 'صاد', 'سنوات', 'سبعمئة', 'اليه', 'هَؤلاء', 'اضحى', 'ذ', 'إياكما', 'اكد', 'لكنَّ', 'أضحى', 'أنه', 'أعلم', 'تانِك', 'وقال', 'كيف', 'أربعة', 'خلال', 'فان', 'ما', 'عاد', 'مليون', 'نا', 'إياه', 'ستكون', 'عدد', 'أبدا', 'ذا', 'رأى', 'تسعون', 'جميع', 'معاذ', 'بئس', 'وأبو', 'سقى', 'دولار', 'طاء', 'نفسها', 'أيضا', 'كن', 'مليم', 'انبرى', 'بسّ', 'امس', 'ى', 'كلتا', 'جدا', 'فلس', 'واكد', 'اطار', 'والتي', 'حيثما', 'تينك', 'منذ', 'نهاية', 'شبه', 'التى', 'ألف', 'ستة', 'ص', 'عيانا', 'لبيك', 'تِه', 'مه', 'هما', 'كاف', 'هنا', 'عنه', 'خاصة', 'كل', 'لديه', 'ومع', 'الذاتي', 'فما', 'برس', 'تلقاء', 'لات', 'ما أفعله', 'علًّ', 'مما', 'بلى', 'سبعون', 'م', 'الحالي', 'أخٌ', 'أمّا', 'أنشأ', 'ديك', 'فعل', 'مهما', 'هى', 'راح', 'الذى', 'ثمَّ', 'تموز', 'كما', 'مر

In [18]:
def remove_stop_words(words, stop_words=stop_words):
    
    result = [] 
    removed_indexes = []
    for i in range(len(words)):
        if words[i] in stop_words:
            removed_indexes.append(i)
        else:
            result.append(words[i])
        
    removed_indexes.sort(reverse=True) # for save remove of pos tagging 
    return result, removed_indexes

remove_stop_words(['هذا', 'هو', 'من', 'الكلمات', 'المستبعدة', 'من', 'التحليل'])

(['الكلمات', 'المستبعدة', 'التحليل'], [5, 2, 1, 0])

Preprocessing: at the beginning we made normlization for the text, then make tokenization for remove the stop words and apply stemming on the words without stop words, 
after that find the pos tagging on the normalized text with stop words to make corrcot results, the remove the pos tags of the stop words, and just save the stemming words with thier pos tagging

In [22]:

# def preprocessing(text):
#     normalization_text = normalization(text)
#     segments = tokenization(normalization_text)

#     # remove stop words from the the text after normalization
#     words, removed_indexes = remove_stop_words(segments)
#     number_of_words = len(words)
    
#     # find the pos tags for the words in the original text after normalization
#     tags = pos_tagging(normalization_text)
   

#     # remove the tags of the stop words
#     for index in removed_indexes:
#         tags.pop(index)

#     # stemming for the words in the original text after tokenization without stop words
#     for i in range(number_of_words):
#         words[i] = stemming(words[i])

#     sample = []

#     # combine the stemming words with their pos tags
#     for i in range(number_of_words):
#         sample.append((words[i], tags[i][1]))

    
#     return sample


def preprocessing(text):
    normalization_text = normalization(text)
    segments = tokenization(normalization_text)

    # remove stop words from the the text after tokenization
    words, _ = remove_stop_words(segments)
    number_of_words = len(words)
    
    
    # stemming for the words in the original text after tokenization without stop words
    for i in range(number_of_words):
        words[i] = stemming(words[i])

 
    return words




text = "أهلاً بكم! هذه الجملة تحتاج للتنظيف. 123"

result = preprocessing(text)
print(result)
for word in result:
    print(word)

['اهل', 'جمل', 'حاج', 'نظف']
اهل
جمل
حاج
نظف


Preprocessing our dataset

In [24]:
for i in range(number_of_samples):
    data_set[i] = (preprocessing(data_set[i][0]), data_set[i][1]) # (words, author)

print(data_set)
for sample in data_set:
    print(sample)
    print()

[(['غرب', 'علم', 'بدع', 'بسط', 'دعو', 'دهش', 'كتب', 'نجب', 'حفظ', 'حلي', 'ريه', 'عال', 'ير', 'علم', 'وضح', 'قهر', 'اين', 'خبئ', 'شبك', 'عملاق', 'حيو', 'وكف', 'فجر', 'تلق', 'وكف', 'نشق', 'هدء', 'هدء', 'ظهر', 'حتم', 'يعط', 'ميز', 'الق', 'خفه', 'عمق'], 'إبراهيم نصر الله'), (['اشر', 'يهو', 'هجم', 'حزب', 'يمن', 'صغر', 'تهم', 'سبب', 'تمك', 'شكل', 'حكم', 'قدم', 'علا', 'حال', 'بزز', 'ثنء', 'سعه', 'بنء', 'حلف', 'نخب', 'طرح', 'طلب', 'بلغ', 'شير', 'يهو', 'يصف', 'حزب', 'بان', 'مثل', 'خطر', 'علا', 'حكم', 'يمن'], 'صالح النعامي'), (['نهي', 'ضمر', 'حيه', 'نتج', 'موت', 'كجز', 'تعب', 'ذكر', 'رور'], 'محمد زينوبة'), (['نسه', 'دين', 'قلن', 'حاش', 'لله', 'علم', 'سوء', 'مكر', 'فتر', 'كذب', 'اصل', 'قلب', 'صدق', 'رفه'], 'محمد زينوبة'), (['وذا', 'حدث', 'كله', 'بيء', 'حدث', 'قرن', 'رغب', 'صحب', 'حتا', 'فشل', 'نزع', 'امر', 'وعص', 'ارا', 'تحب', 'نكم', 'يرد', 'دنا', 'ومن', 'يرد', 'اخر', 'خطب', 'قرا', 'ثري', 'يجب', 'عمل', 'بجد', 'واستبصار'], 'معتز خطيب'), (['سدر', 'قئل', 'علا', 'رغم', 'يهو', 'عني', 'بعد', 'رائيل', '

Features Extraction

In [37]:
# TF-IDF
def tf_idf(corpus):
    from sklearn.feature_extraction.text import TfidfVectorizer


    # Extract the sentences from the corpus
    sentences = [sample[0] for sample in corpus]

    # Create a list of authors
    authors = [sample[1] for sample in corpus]

    # Convert the sentences to strings
    sentences = [' '.join(sentence) for sentence in sentences]

    # Create the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the sentences
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Get the feature names (unique words) as columns
    feature_names = vectorizer.get_feature_names()

    # Print the feature names
    print("Feature Names:", feature_names)
    print("Number of Features:", len(feature_names))

    # Print the TF-IDF matrix
    print("TF-IDF Matrix:")
    print(tfidf_matrix.toarray())
    
    # Print the first row of the TF-IDF matrix
    print(tuple(tfidf_matrix.toarray()[0]))

    # Print the author names
    print("Authors:", authors)

    return tfidf_matrix.toarray(), feature_names, authors

        
print(f'len: {len(data_set)}')
tf_idf(data_set)

len: 1594
Feature Names: ['ؤثر', 'ؤجج', 'ؤجل', 'ؤخر', 'ؤده', 'ؤرق', 'ؤسء', 'ؤسس', 'ؤسف', 'ؤشر', 'ؤقت', 'ؤكد', 'ؤكل', 'ؤلف', 'ؤلم', 'ؤمر', 'ؤمن', 'ؤنث', 'ؤنس', 'ؤهل', 'ؤون', 'ؤيد', 'ئات', 'ئتم', 'ئلف', 'ئما', 'ئوه', 'ائت', 'ائتلافيه', 'ائتمانيه', 'ائم', 'ائه', 'االتاريخ', 'اب', 'ابا', 'ابارتهايد', 'ابب', 'ابت', 'ابتسامته', 'ابح', 'ابد', 'ابر', 'ابريلنيس', 'ابض', 'ابع', 'ابق', 'ابك', 'ابل', 'ابن', 'ابه', 'ابو', 'ابي', 'ات', 'اتا', 'اتالانتا', 'اتت', 'اتج', 'اتح', 'اتر', 'اتف', 'اتق', 'اتل', 'اتم', 'اتن', 'اتو', 'اتي', 'اثب', 'اثخ', 'اثر', 'اثل', 'اثم', 'اثن', 'اثنياتالملل', 'اثو', 'اجئ', 'اجب', 'اجتماعات', 'اجتماعيه', 'اجتماعيهالقاعده', 'اجتهاديه', 'اجج', 'اجد', 'اجدادكالمتجذره', 'اجر', 'اجل', 'اجه', 'احا', 'احب', 'احت', 'احتجاجيه', 'احتكاريه', 'احتياجات', 'احتياطيه', 'احج', 'احد', 'احذ', 'احز', 'احس', 'احص', 'احض', 'احط', 'احق', 'احك', 'احل', 'احم', 'احه', 'احي', 'اخا', 'اخت', 'اختباراته', 'اختصاصيه', 'اختلافات', 'اختياراته', 'اخد', 'اخذ', 'اخر', 'اخراللاجئ', 'اخرالمجتمع', 'اخرالموت', '

C:\Users\swata\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 ['ؤثر',
  'ؤجج',
  'ؤجل',
  'ؤخر',
  'ؤده',
  'ؤرق',
  'ؤسء',
  'ؤسس',
  'ؤسف',
  'ؤشر',
  'ؤقت',
  'ؤكد',
  'ؤكل',
  'ؤلف',
  'ؤلم',
  'ؤمر',
  'ؤمن',
  'ؤنث',
  'ؤنس',
  'ؤهل',
  'ؤون',
  'ؤيد',
  'ئات',
  'ئتم',
  'ئلف',
  'ئما',
  'ئوه',
  'ائت',
  'ائتلافيه',
  'ائتمانيه',
  'ائم',
  'ائه',
  'االتاريخ',
  'اب',
  'ابا',
  'ابارتهايد',
  'ابب',
  'ابت',
  'ابتسامته',
  'ابح',
  'ابد',
  'ابر',
  'ابريلنيس',
  'ابض',
  'ابع',
  'ابق',
  'ابك',
  'ابل',
  'ابن',
  'ابه',
  'ابو',
  'ابي',
  'ات',
  'اتا',
  'اتالانتا',
  'اتت',
  'اتج',
  'اتح',
  'اتر',
  'اتف',
  'اتق',
  'اتل',
  'اتم',
  'اتن',
  'اتو',
  'اتي',
  'اثب',
  'اثخ',
  'اثر',
  'اثل',
  'اثم',
  'اثن',
  'اثنياتالملل',
  'اثو',
  'اجئ',
  'اجب',
  'اجتماعات',
  'اجتماعيه

In [45]:
# Word Embedding
def word_embedding(corpus):
    from gensim.models import FastText
    import numpy as np

    # Extract the sentences from the corpus
    sentences = [sample[0] for sample in corpus]

    # Create a list of authors
    authors = [sample[1] for sample in corpus]

    # Train FastText model
    model = FastText(sentences, vector_size=100, window=5, min_count=2, workers=4, sg=0)

    # Create feature vectors for each sentence
    feature_vectors = []
    for sentence in sentences:
        vectors = [model.wv[word] for word in sentence if word in model.wv]
        if vectors:
            sentence_vector = np.mean(vectors, axis=0)
        else:
            sentence_vector = np.zeros(model.vector_size)
        feature_vectors.append(sentence_vector)

    # Convert feature vectors to NumPy array
    feature_vectors = np.array(feature_vectors)

    # Print the feature vectors
    print("Feature Vectors:")
    print(feature_vectors)

    # Print the first feature vector
    print("First Feature Vector:", feature_vectors[0].tolist())

    # Print the author names
    print("Authors:", authors)

    return feature_vectors, authors

word_embedding(data_set)


Feature Vectors:
[[ 0.05476921  0.01015007  0.27762839 ...  0.21067421 -0.10042796
   0.14600889]
 [ 0.06747236  0.01172782  0.34954032 ...  0.26641655 -0.12645003
   0.18365271]
 [ 0.06714171  0.0106847   0.33839601 ...  0.25655663 -0.12337538
   0.17856927]
 ...
 [ 0.06563491  0.01118178  0.3356528  ...  0.25518209 -0.12202015
   0.17628674]
 [ 0.06311983  0.01163996  0.33175609 ...  0.25344053 -0.11927558
   0.17392315]
 [ 0.06233883  0.01069316  0.32130572 ...  0.24369772 -0.11599281
   0.16867112]]
First Feature Vector: [0.054769206792116165, 0.010150069370865822, 0.2776283919811249, 0.26757746934890747, -0.2570570409297943, -0.17146286368370056, -0.07386177033185959, 0.4297122657299042, 0.10416296869516373, -0.32745641469955444, -0.06754059344530106, -0.4026463031768799, -0.4485575258731842, 0.2815000116825104, 0.030048653483390808, 0.267802357673645, -0.14165440201759338, -0.5125131607055664, -0.18876101076602936, 0.14948773384094238, -0.05605247616767883, 0.5098247528076172, -0

(array([[ 0.05476921,  0.01015007,  0.27762839, ...,  0.21067421,
         -0.10042796,  0.14600889],
        [ 0.06747236,  0.01172782,  0.34954032, ...,  0.26641655,
         -0.12645003,  0.18365271],
        [ 0.06714171,  0.0106847 ,  0.33839601, ...,  0.25655663,
         -0.12337538,  0.17856927],
        ...,
        [ 0.06563491,  0.01118178,  0.3356528 , ...,  0.25518209,
         -0.12202015,  0.17628674],
        [ 0.06311983,  0.01163996,  0.33175609, ...,  0.25344053,
         -0.11927558,  0.17392315],
        [ 0.06233883,  0.01069316,  0.32130572, ...,  0.24369772,
         -0.11599281,  0.16867112]]),
 ['إبراهيم نصر الله',
  'صالح النعامي',
  'محمد زينوبة',
  'محمد زينوبة',
  'معتز خطيب',
  'صالح النعامي',
  'عبسي سميسم',
  'إياد حميد',
  'معتز خطيب',
  'نضال محمد وتد',
  'إبراهيم نصر الله',
  'صالح النعامي',
  'إبراهيم نصر الله',
  'صالح النعامي',
  'عبير قلوطي',
  'معتز خطيب',
  'عبد الله الشريف',
  'يونس جعدي',
  'نضال محمد وتد',
  'إبراهيم نصر الله',
  'يونس جعدي'